In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append('../src')
import const
from utils import DataHandler, Timer

In [2]:
dh = DataHandler()
t = Timer()

In [3]:
# train_df = pd.read_csv(const.INPUT_DATA_DIR / 'train.csv', dtype=const.DTYPE, nrows=10**2)

questions_df = pd.read_csv('../data/input/questions.csv')
questions_df['tags'] = questions_df['tags'].fillna(0)
questions_df['tag_list'] = questions_df['tags'].apply(lambda tags: [int(tag) for tag in str(tags).split(' ')])
questions_df['tag_list'] = questions_df['tag_list'].apply(lambda x: [188] * (6 - len(x)) + x)
q2tg = dict(questions_df[['question_id', 'tag_list']].values)
tag_array = np.array(list(questions_df['tag_list'].values))

In [4]:
seq_list = dh.load('../data/seq10/row_22731.pkl')
target_id = seq_list[0]

In [5]:
%%time
difftime = np.array(seq_list[1]) / 60_000.   # ms -> m
difftime = np.where(difftime < 0, 300, difftime)
difftime = np.log1p(difftime)

CPU times: user 50 µs, sys: 7 µs, total: 57 µs
Wall time: 46.5 µs


In [6]:
%%time
prior_elapsed = np.array(seq_list[2]) / 1_000.
prior_elapsed = np.log1p(prior_elapsed)
prior_elapsed = np.where(np.isnan(prior_elapsed), np.log1p(21), prior_elapsed)

CPU times: user 39 µs, sys: 5 µs, total: 44 µs
Wall time: 39.1 µs


In [7]:
content_id = seq_list[0]
exp = seq_list[3]
target = seq_list[5]
te_content_id = seq_list[6]

In [8]:
%%time
avg_u_target = np.zeros(120, dtype=float)
start_idx = np.where(content_id > 0)[0][0]
ac_latest = target[start_idx:]
avg_u_target[start_idx:] = ac_latest.cumsum() / (np.arange(len(ac_latest)) + 1)

CPU times: user 73 µs, sys: 10 µs, total: 83 µs
Wall time: 63.4 µs


In [9]:
%%time

exp = np.where(np.isnan(exp), 0, exp)
te_content_id = np.where(np.isnan(te_content_id), 0.625164097637492, te_content_id)   # nanmean
avg_u_target = np.where(np.isnan(avg_u_target), 0, avg_u_target)
num_feat = np.vstack([te_content_id, avg_u_target]).T

CPU times: user 52 µs, sys: 7 µs, total: 59 µs
Wall time: 54.1 µs


In [10]:
%%time

qtg1 = np.zeros((120, 6)) + 188
for idx, cid in enumerate(content_id[start_idx:]):   # start_idx = np.where(seq_content_id > 0)[0][0]
    qtg1[start_idx + idx, :] = q2tg[cid]

CPU times: user 253 µs, sys: 34 µs, total: 287 µs
Wall time: 278 µs


In [11]:
%%time

qtg2 = np.zeros((120, 6)) + 188
tag_seq_list = []
for idx, cid in enumerate(content_id[start_idx:]):   # start_idx = np.where(seq_content_id > 0)[0][0]
    a.append(q2tg[cid])
    
qtg2[start_idx:, :] = a

CPU times: user 223 µs, sys: 0 ns, total: 223 µs
Wall time: 214 µs


In [12]:
b = np.array(list(q2tg.values()))

In [13]:
%%time

qtg3 = np.zeros((120, 6)) + 188
    
qtg3[start_idx:, :] = tag_array[content_id[start_idx:]]

CPU times: user 105 µs, sys: 0 ns, total: 105 µs
Wall time: 75.8 µs


In [14]:
46.3 + 38.6 + 65.1 + 52.9

202.9